# 函数进阶

## 函数是基本类型

在 `Python` 中，函数是一种基本类型的对象，这意味着

* 可以将函数作为参数传给另一个函数
* 将函数作为字典的值储存
* 将函数作为另一个函数的返回值

In [5]:
def square(x):
    """Square of x."""
    return x*x

def cube(x):
    """Cube of x."""
    return x*x*x

作为字典的值：

In [2]:
funcs = {
    'square': square,
    'cube': cube,}

x = 2
print(square(x))
print(cube(x))

for func in sorted(funcs):
    print(func, funcs[func](x))

4
8
cube 8
square 4


## 函数参数

### 引用传递

`Python` 中的函数传递方式是 `call by reference` 即引用传递，例如，对于这样的用法：

    x = [10, 11, 12]
    f(x)

传递给函数 `f` 的是一个指向 `x` 所包含内容的引用，如果我们修改了这个引用所指向内容的值（例如 `x[0]=999`），那么外面的 `x` 的值也会被改变。不过如果我们在函数中赋给 `x` 一个新的值（例如另一个列表），那么在函数外面的 `x` 的值不会改变：

In [3]:
def mod_f(x):
    x[0] = 999
    return x

x = [1, 2, 3]

print(x)
print(mod_f(x))
print(x)

[1, 2, 3]
[999, 2, 3]
[999, 2, 3]


In [4]:
def no_mod_f(x):
    x = [4, 5, 6]
    return x

x = [1,2,3]

print(x)
print(no_mod_f(x))
print(x)

[1, 2, 3]
[4, 5, 6]
[1, 2, 3]


上面两种不同的原因在【python基本编程】章节有具体介绍。

### 默认参数是可变的！

函数可以传递默认参数，默认参数的绑定发生在函数定义的时候，以后每次调用默认参数时都会使用同一个引用。

这样的机制会导致这种情况的发生：

In [10]:
def f(x = []):
    x.append(1)
    return x

理论上说，我们希望调用 `f()` 时返回的是 `[1]`， 但事实上：

In [11]:
print(f())
print(f())
print(f())
print(f(x = [9,9,9]))
print(f())
print(f())

[1]
[1, 1]
[1, 1, 1]
[9, 9, 9, 1]
[1, 1, 1, 1]
[1, 1, 1, 1, 1]


而我们希望看到的应该是这样：

In [12]:
def f(x = None):
    if x is None:
        x = []
    x.append(1)
    return x

print(f())
print(f())
print(f())
print(f(x = [9,9,9]))
print(f())
print(f())

[1]
[1]
[1]
[9, 9, 9, 1]
[1]
[1]


## 高阶函数

以函数作为参数，或者返回一个函数的函数是高阶函数，常用的例子有 `map` 和 `filter` 函数：

`map(f, sq)` 函数将 `f` 作用到 `sq` 的每个元素上去，并返回结果组成的列表，相当于：
```python
[f(s) for s in sq]
```

注：python3中返回的不是列表，是一个迭代器，所以需要在外面加一层`list()`才会得到列表。

In [17]:
list(map(square, range(5)))

[0, 1, 4, 9, 16]

`filter(f, sq)` 函数的作用相当于，对于 `sq` 的每个元素 `s`，返回所有 `f(s)` 为 `True` 的 `s` 组成的列表，相当于：
```python
[s for s in sq if f(s)]
```

In [19]:
def is_even(x):
    return x % 2 == 0

list(filter(is_even, range(5)))

[0, 2, 4]

一起使用：

In [21]:
list(map(square, filter(is_even, range(5))))

[0, 4, 16]

`reduce(f, sq)` 函数接受一个二元操作函数 `f(x,y)`，并对于序列 `sq` 每次合并两个元素：

In [23]:
from functools import reduce
def my_add(x, y):
    return x + y

reduce(my_add, [1,2,3,4,5])

15

传入加法函数，相当于对序列求和。

返回一个函数：

In [24]:
def make_logger(target):
    def logger(data):
        with open(target, 'a') as f:
            f.write(data + '\n')
    return logger

foo_logger = make_logger('foo.txt')
foo_logger('Hello')
foo_logger('World')

In [30]:
import os
os.remove('foo.txt')

## 匿名函数

在使用 `map`， `filter`，`reduce` 等函数的时候，为了方便，对一些简单的函数，我们通常使用匿名函数的方式进行处理，其基本形式是：

    lambda <variables>: <expression>

例如，我们可以将这个：

In [32]:
print(list(map(square, range(5))))

[0, 1, 4, 9, 16]


用匿名函数替换为：

In [33]:
print(list(map(lambda x: x * x, range(5))))

[0, 1, 4, 9, 16]


匿名函数虽然写起来比较方便（省去了定义函数的烦恼），但是有时候会比较难于阅读：

In [34]:
s1 = reduce(lambda x, y: x+y, map(lambda x: x**2, range(1,10)))
print(s1)

285


当然，更简单地，我们可以写成这样：

In [35]:
s2 = sum(x**2 for x in range(1, 10))
print(s2)

285


# 作用域

在函数中，`Python` 从命名空间中寻找变量的顺序如下：

- `local function scope`：局部
- `enclosing scope`：嵌套
- `global scope`：全局
- `builtin scope`：内建

例子：

## local 作用域

In [1]:
def foo(a,b):
    c = 1
    d = a + b + c

这里所有的变量都在 `local` 作用域。

## global 作用域与关键词

In [2]:
c = 1
def foo(a,b):
    d = a + b + c

这里的 `c` 就在 `global` 作用域。

使用 `global` 关键词可以在 `local` 作用域中修改 `global` 作用域的值。

In [4]:
x = 15

def print_newx():
    global x
    x = 18

print(x)
print_newx()
print(x)

15
18


其作用是将 `x` 指向 `global` 中的 `x`。

如果不加关键词，那么 `local` 作用域的 `x` 不会影响 `global` 作用域中的值：

In [5]:
x = 15

def print_newx():
    x = 18
    
print_newx()

print(x)

15


## built-in 作用域

In [7]:
def list_length(a):
    return len(a)

a = [1,2,3]
print( list_length(a))

3


这里函数 `len` 就是在 `built-in` 作用域中：

In [10]:
import builtins
builtins.len

<function len>

## 词法作用域

对于嵌套函数：

In [14]:
def outer():
    a = 1
    def inner():
        print ("a =", a)
    return inner()

outer()

a = 1


如果里面的函数没有找到变量，那么会向外一层寻找变量，如果再找不到，则到 `global` 作用域。

返回的是函数时的情况：

In [13]:
def outer():
    a = 1
    def inner():
        return a
    return inner
    
func = outer()

print ('a (1):', func())

a (1): 1


func() 函数中调用的 `a` 要从它定义的地方开始寻找，而不是在 `func` 所在的作用域寻找。

# 递归

**闭包**指延伸了作用域的函数，其中包含在函数定义体重引用、但是不在定义体中定义的**非全局变量**。也就是说，它能访问定义体之外定义的非全局变量。

* 只有在涉及**嵌套函数**时才有闭包问题；
* 这里提到的非全局变量也就是指 嵌套作用域`enclosing scope`。


Fibocacci 数列：

In [39]:
def fib1(n):
    """Fib with recursion."""

    # base case
    if n==0 or n==1:
        return 1
    # recurssive caae
    else:
        return fib1(n-1) + fib1(n-2)

print([fib1(i) for i in range(10)])

[1, 1, 2, 3, 5, 8, 13, 21, 34, 55]


一个更高效的非递归版本：

In [40]:
def fib2(n):
    """Fib without recursion."""
    a, b = 0, 1
    for i in range(1, n+1):
        a, b = b, a+b
    return b

print([fib2(i) for i in range(10)])

[1, 1, 2, 3, 5, 8, 13, 21, 34, 55]


速度比较：

In [41]:
%timeit fib1(20)
%timeit fib2(20)

4.42 ms ± 132 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
1.76 µs ± 75.1 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


对于第一个递归函数来说，调用 `fib(n+2)` 的时候计算 `fib(n+1), fib(n)`，调用 `fib(n+1)` 的时候也计算了一次 `fib(n)`，这样造成了重复计算。

使用缓存机制的递归版本，这里利用了默认参数可变的性质，构造了一个缓存：

In [42]:
def fib3(n, cache={0: 1, 1: 1}):
    """Fib with recursion and caching."""

    try:
        return cache[n]
    except KeyError:
        cache[n] = fib3(n-1) + fib3(n-2)
        return cache[n]

print ([fib3(i) for i in range(10)])

%timeit fib1(20)
%timeit fib2(20)
%timeit fib3(20)

[1, 1, 2, 3, 5, 8, 13, 21, 34, 55]
4.38 ms ± 69.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
1.74 µs ± 58.3 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
170 ns ± 6.95 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


# 迭代器

迭代是数据处理的基石。扫描内存中放不下的数据集时，我们要找到一种惰性获取数据项的方式，即按需一次获取一个数据项。这就是迭代器模式（Iterator pattern）。

在 Python 中，所有集合都可以迭代。在 Python 语言内部，迭代器用于支持：
* for 循环
* 构建和扩展集合类型
* 逐行遍历文本文件
* 列表推导、字典推导和集合推导
* 元组拆包
* 调用函数时，使用 * 拆包实参

## iter函数

序列可以迭代的原因：`iter函数`，解释器需要迭代对象 x 时，会自动调用 `iter(x)`。

内置的 `iter 函数`有以下作用:
1. 检查对象是否实现了 `__iter__` 方法，如果实现了就调用它，获取一个迭代器;
2. 如果没有实现 `__iter__` 方法，但是实现了 `__getitem__` 方法，Python 会创建一个迭代器，尝试按顺序（从索引 0 开始）获取元素。
3. 如果尝试失败，Python 抛出 TypeError 异常，通常会提示`“C object is not iterable”（C 对象不可迭代）`，其中 C 是目标对象所属的类。

任何 Python 序列都可迭代的原因是，它们都实现了 `__getitem__` 方法。其实，标准的序列也都实现了 `__iter__` 方法，因此如果你想创建一个迭代序列的时候，也应该实现这个方法。

之所以对 `__getitem__` 方法做特殊处理，是为了向后兼容。

***
下面举一个实现了`__getitem__`方法然后可以迭代的例子。

In [2]:
import re
import reprlib
RE_WORD = re.compile('\w+')

class Sentence:
    def __init__(self, text):
        self.text = text
        self.words = RE_WORD.findall(text) 
        
    def __getitem__(self, index):
        return self.words[index] 
    
    def __len__(self): 
        return len(self.words)
    
    def __repr__(self):
        return 'Sentence(%s)' % reprlib.repr(self.text)

备注：
1. 为了完善序列协议，我们实现了 `__len__` 方法；不过，为了让对象可以迭代，没必要实现这个方法。
2. `reprlib.repr` 这个实用函数用于生成大型数据结构的简略字符串表示形式。

In [3]:
#测试是否可以迭代
s = Sentence('"The time has come," the Walrus said,') 
s

Sentence('"The time ha... Walrus said,')

In [4]:
for word in s: 
    print(word)

The
time
has
come
the
Walrus
said


In [5]:
list(s)

['The', 'time', 'has', 'come', 'the', 'Walrus', 'said']

***

In [6]:
from collections import abc
issubclass(Sentence, abc.Iterable)

False

备注：
1. 我们定义的Sentence 类是可以迭代的，但却无法通过 `issubclass (Sentence, abc.Iterable)` 测试，因为它不是标准的迭代器，即没有实现`__iter__`方法；
2. 从 Python 3.4 开始，检查对象 x 能否迭代，最准确的方法是：

    调用 `iter(x)` 函数，如果不可迭代，再处理 TypeError 异常。这比使用 `isinstance(x, abc.Iterable)` 更准确，因为`iter(x)`函数会考虑到遗留的 `__getitem__` 方法，而 `abc.Iterable` 类则
    不考虑。

## 可迭代的对象与迭代器

**可迭代的对象**：

* 使用 **iter 内置函数**可以获取**迭代器**的对象。如果对象实现了能返回迭代器的 `__iter__` 方法，那么对象就是可迭代的。序列都可以迭代；
    
* 实现了 `__getitem__` 方法，而且其参数是从零开始的索引，这种对象也可以迭代。

可迭代的对象和迭代器之间的关系：**Python 从可迭代的对象中获取迭代器.**

标准的**迭代器接口**有两个方法:
* `__next__`:返回下一个可用的元素，如果没有元素了，抛出 StopIteration异常。
* `__iter__`:返回 self，以便在应该使用可迭代对象的地方使用迭代器，例如在 for 循环中。

这个接口在 `collections.abc.Iterator` 抽象基类中制定:

* 这个类定义了 `__next__` 抽象方法，而且继承自 `Iterable` 类；

* `__iter__` 抽象方法则在 `Iterable` 类中定义。

如下图所示：

<img src='input_data/迭代类.png'>

下面我们重构`Sentence`类，按照严格的迭代器接口定义以及可迭代对象定义来实现。

In [11]:
import re
import reprlib
RE_WORD = re.compile('\w+')
class Sentence:
    def __init__(self, text):
        self.text = text
        self.words = RE_WORD.findall(text)
    
    def __repr__(self):
        return 'Sentence(%s)' % reprlib.repr(self.text)

    def __iter__(self): 
        return SentenceIterator(self.words)

class SentenceIterator:
    def __init__(self, words):
        self.words = words 
        self.index = 0 
        
    def __next__(self):
        try:
            word = self.words[self.index] 
        except IndexError:
            raise StopIteration() 
        self.index += 1 
        return word

    def __iter__(self): 
        return self

我们发现，使用标准的迭代器接口定义实现的迭代器对象可以通过`issubclass`的测试：

In [12]:
from collections import abc
issubclass(Sentence, abc.Iterable)

True

**备注：**
可迭代的对象一定不能是自身的迭代器。也就是说，可迭代的对象必须实现 `__iter__` 方法，但不能实现 `__next__` 方法。

## 生成器函数yield

* 只要 Python 函数的定义体中有 yield 关键字，该函数就是**生成器函数**。调用生成器函数时，会返回一个**生成器对象**。也就是说，生成器函数是生成器工厂。

* 普通的函数与生成器函数在句法上唯一的区别是，在后者的定义体中有 yield 关键字。

下面举一个简单的例子来说明生成器函数。

In [13]:
def gen_123():
    yield 1
    yield 2
    yield 3

In [14]:
gen_123

<function __main__.gen_123()>

In [15]:
gen_123()

<generator object gen_123 at 0x0000022BC97EC228>

我们注意到：`gen_123` 是一个函数对象，但是调用时 `gen_123()` 返回一个生成器对象。

In [21]:
# 生成器是迭代器，会生成传给 `yield` 关键字的表达式的值
for i in gen_123(): 
    print(i)

1
2
3


In [17]:
g=gen_123()
next(g)

1

In [18]:
next(g)

2

In [19]:
next(g)

3

In [22]:
next(g)

StopIteration: 

生成器函数会创建一个生成器对象，把生成器传给 `next(...)` 函数时，生成器函数会向前，执行函数定义体中的下一个 yield 语句，返回产出的值，并在函数定义体的当前位置暂停。最终，函数的定义体返回时，外层的生成器对象会抛出
StopIteration 异常——这一点与迭代器协议一致。

下面使用生成器函数来实现 Sentence类：

In [1]:
import re
import reprlib
RE_WORD = re.compile('\w+')
class Sentence:
    def __init__(self, text):
        self.text = text
        self.words = RE_WORD.findall(text)
    def __repr__(self):
        return 'Sentence(%s)' % reprlib.repr(self.text)
    def __iter__(self):
        for word in self.words: 
            yield word 
            return 

我们会发现使用生成器函数的好处在于，不用再单独定义一个迭代器类。

### 惰性实现Sentence类

目前实现的几版 Sentence 类都不具有惰性，因为 `__init__` 方法急迫地构建好了文本中的单词列表，然后将其绑定到 `self.words` 属性上。这样就得处理整个文本，列表使用的内存量可能与文本本身一样多（或许更多，这取决于文本中有多少非单词字符）。如果只需迭代前几个单词，大多数工作都是白费力气。

`re.finditer` 函数是 `re.findall` 函数的惰性版本，返回的不是列表，而是一个生成器，按需生成 `re.MatchObject` 实例。如果有很多匹配，`re.finditer`  函数能节省大量内存。我们要使用这个函数让下一版的 Sentence 类变得懒惰，即只在需要时才生成下一个单词。

In [2]:
import re
import reprlib
RE_WORD = re.compile('\w+')
class Sentence:
    def __init__(self, text):
        self.text = text 
    def __repr__(self):
        return 'Sentence(%s)' % reprlib.repr(self.text)
    def __iter__(self):
        for match in RE_WORD.finditer(self.text): 
            yield match.group() 

## 生成器表达式

* 生成器函数已经极大地简化了代码，但是使用生成器表达式甚至能把代码变得更简短；
* 生成器表达式可以理解为**列表推导**的惰性版本：不会迫切地构建列表，而是返回一个生成器，按需惰性生成元素。如果列表推导是制造列表的工厂，那么生成器表达式就是制造生成器的工厂。

使用生成器表达式实现 Sentence类：

In [4]:
import re
import reprlib
RE_WORD = re.compile('\w+')
class Sentence:
    def __init__(self, text):
        self.text = text 
    def __repr__(self):
        return 'Sentence(%s)' % reprlib.repr(self.text)
    def __iter__(self):
        return (match.group() for match in RE_WORD.finditer(self.text))

## 生成器

**迭代器与生成器**：
1. 所有生成器都是迭代器，因为生成器完全实现了迭代器接口；
2. 迭代器用于从集合中取出元素；而生成器用于“凭空”生成元素。

通过斐波纳契数列能很好地说明二者之间的区别：斐波纳契数列中的数有无穷个，在一个集合里放不下。

不过在 Python社区中，大多数时候都把迭代器和生成器视作同一概念。

In [62]:
def fibonacci():
    a, b = 0, 1
    while True:
        yield a
        a, b = b, a + b

In [63]:
next(fibonacci())

0

## 标准库中的生成器函数

### 用于过滤

* `itertools.compress(it,selector_it)`：并行处理两个可迭代的对象；如果 selector_it中的元素是真值，产出 it 中对应的元素

In [3]:
import itertools
list(itertools.compress(range(7),[0,1,0,1]))

[1, 3]

* `itertools.dropwhile(predicate,it)`：跳过 predicate 的计算结果为真值的元素，然后产出剩下的各个元素（不再进一步检查）;

* `itertools.takewhile(predicate,it)`：predicate 返回真值时产出对应的元素，然后立即停止，不再继续检查

In [5]:
def vowel(c):
    return c.lower() in 'aeiou'

In [6]:
list(itertools.dropwhile(vowel,'Aardvark'))

['r', 'd', 'v', 'a', 'r', 'k']

In [11]:
list(itertools.takewhile(vowel, 'Aardvark'))

['A', 'a']

* `filter(predicate, it)`(内置):把 it 中的各个元素传给 predicate，如果predicate(item) 返回真值，那么产出对应的元
素；如果 predicate 是 None，那么只产出真值元素;

* `itertools.filterfalse(predicate,it)`:与 `filter`函数的作用类似，不过 predicate 的逻辑是相反的：predicate 返回假值时产出对应
的元素

In [8]:
list(filter(vowel, 'Aardvark'))

['A', 'a', 'a']

In [9]:
list(itertools.filterfalse(vowel, 'Aardvark'))

['r', 'd', 'v', 'r', 'k']

* `itertools.islice(it, stop,step=1)`:产出 it 的切片，作用类似于 `s[:stop]` 或`s[start:stop:step]`，不过 it 可以是任何可迭代
的对象，而且这个函数实现的是惰性操作

In [10]:
list(itertools.islice('Aardvark', 1, 7, 2))

['a', 'd', 'a']

### 用于映射

* `itertools.accumulate(it,[func])`：产出累积的总和；如果提供了 func，那么把前两个元素传给它，然后把计算结果和下一个元素传给
它，以此类推，最后产出结果

In [12]:
list(itertools.accumulate(range(7)))

[0, 1, 3, 6, 10, 15, 21]

In [13]:
sample = [5, 4, 2, 8, 7, 6, 3, 0, 9, 1]
list(itertools.accumulate(sample, max))

[5, 5, 5, 8, 8, 8, 8, 8, 9, 9]

* `enumerate(iterable,start=0)`：产出由两个元素组成的元组，结构是 (index,item)，其中 index 从 start 开始计数，item 则从
iterable 中获取

In [15]:
list(enumerate(range(7),start=1))

[(1, 0), (2, 1), (3, 2), (4, 3), (5, 4), (6, 5), (7, 6)]

* `map(func, it1,[it2, ..., itN])`：把 it 中的各个元素传给func，产出结果；如果传入N 个可迭代的对象，那么 func 必须能接受 N 个参
数，而且要并行处理各个可迭代的对象

In [16]:
list(map(lambda a, b: (a, b), range(11), [2, 4, 8]))

[(0, 2), (1, 4), (2, 8)]

* `itertools.starmap(func, it)`：把 it 中的各个元素传给 func，产出结果；输入的可迭代对象应该产出可迭代的元素 iit，然后以func(*iit) 这种形式调用 func

In [19]:
import operator
list(itertools.starmap(operator.mul, enumerate('albatroz', 1)))

['a', 'll', 'bbb', 'aaaa', 'ttttt', 'rrrrrr', 'ooooooo', 'zzzzzzzz']

### 合并多个可迭代对象

* `itertools chain(it1, ..., itN)`：先产出 it1 中的所有元素，然后产出 it2 中的所有元素，以此类推，无缝连接在一起;

* `itertools.chain.from_iterable(it)`：产出 it 生成的各个可迭代对象中的元素，一个接一个，无缝连接在一起；it 应该产出可迭代的元素，例如可迭代的对象列表

In [20]:
list(itertools.chain('ABC', range(2)))

['A', 'B', 'C', 0, 1]

In [21]:
list(itertools.chain.from_iterable(enumerate('ABC')))

[0, 'A', 1, 'B', 2, 'C']

* `itertools.product(it1, ..., itN,repeat=1)`：计算笛卡儿积：从输入的各个可迭代对象中获取元素，合并成由 N 个元素组成的元组，与嵌套的 for 循环效果一样；repeat 指明重复处理多少次输入的可迭代对象

In [27]:
list(itertools.product('ABC', range(2)))

[('A', 0), ('A', 1), ('B', 0), ('B', 1), ('C', 0), ('C', 1)]

In [28]:
list(itertools.product('ABC', repeat=2))

[('A', 'A'),
 ('A', 'B'),
 ('A', 'C'),
 ('B', 'A'),
 ('B', 'B'),
 ('B', 'C'),
 ('C', 'A'),
 ('C', 'B'),
 ('C', 'C')]

In [29]:
rows = itertools.product('AB', range(2), repeat=2)
for row in rows: 
    print(row)

('A', 0, 'A', 0)
('A', 0, 'A', 1)
('A', 0, 'B', 0)
('A', 0, 'B', 1)
('A', 1, 'A', 0)
('A', 1, 'A', 1)
('A', 1, 'B', 0)
('A', 1, 'B', 1)
('B', 0, 'A', 0)
('B', 0, 'A', 1)
('B', 0, 'B', 0)
('B', 0, 'B', 1)
('B', 1, 'A', 0)
('B', 1, 'A', 1)
('B', 1, 'B', 0)
('B', 1, 'B', 1)


* `zip(it1, ..., itN)`：并行从输入的各个可迭代对象中获取元素，产出由 N 个元素组成的元组，只要有一个可迭代的对象到头了，就默默地停止；

* `itertools.zip_longest(it1, ...,itN, fillvalue=None)`：并行从输入的各个可迭代对象中获取元素，产出由 N 个元素组成的元组，等到最长的可迭代对象到头后才停止，空缺的值使用 fillvalue填充

In [30]:
list(zip('ABC', range(5), [10, 20, 30, 40]))

[('A', 0, 10), ('B', 1, 20), ('C', 2, 30)]

In [32]:
list(map(lambda x,y,z:(x,y,z),'ABC', range(5), [10, 20, 30, 40]))

[('A', 0, 10), ('B', 1, 20), ('C', 2, 30)]

In [33]:
list(itertools.zip_longest('ABC', range(5)))

[('A', 0), ('B', 1), ('C', 2), (None, 3), (None, 4)]

In [34]:
list(itertools.zip_longest('ABC', range(5), fillvalue='?'))

[('A', 0), ('B', 1), ('C', 2), ('?', 3), ('?', 4)]

### 扩展元素

* `itertools.combinations(it, out_len)`: 把 it 产出的 out_len 个元素组合在一起，然后产出;

* `itertools.combinations_with_replacement(it,out_len)`: 把 it 产出的 out_len 个元素组合在一起，然后产出，包含相同元素的组合;

* `itertools.permutations(it, out_len=None)`: 把 out_len 个 it 产出的元素排列在一起，然后产出这些排列；out_len的默认值等于 len(list(it));

In [35]:
list(itertools.combinations('ABC', 2))

[('A', 'B'), ('A', 'C'), ('B', 'C')]

In [36]:
list(itertools.combinations_with_replacement('ABC', 2))

[('A', 'A'), ('A', 'B'), ('A', 'C'), ('B', 'B'), ('B', 'C'), ('C', 'C')]

In [40]:
list(itertools.permutations('ABC', 2))

[('A', 'B'), ('A', 'C'), ('B', 'A'), ('B', 'C'), ('C', 'A'), ('C', 'B')]

* `itertools.count(start=0, step=1)`: 从 start 开始不断产出数字，按step 指定的步幅增加;
* `itertools.cycle(it)`: 从 it 中产出各个元素，存储各个元素的副本，然后按顺序重复不断地产出各个元素;
* `itertools.repeat(item, [times])`: 重复不断地产出指定的元素，除非提供 times，指定次数;

备注：前两个生成器只有受到 islice 函数的限制，才能构建列表，否则会变成一个无穷列表

In [37]:
list(itertools.islice(itertools.count(1, .3), 3))

[1, 1.3, 1.6]

In [38]:
list(itertools.islice(itertools.cycle('ABC'), 7))

['A', 'B', 'C', 'A', 'B', 'C', 'A']

In [39]:
list(itertools.repeat(8, 4))

[8, 8, 8, 8]

### 用于重新排列元素

* `itertools.groupby(it,key=None)`:产出由两个元素组成的元素，形式为 (key,group)，其中 key 是分组标准，group 是生成器，用于产出分组里的元素;
* `reversed(seq)`:从后向前，倒序产出 seq 中的元素；seq 必须是序列，或者是实现了 __reversed__ 特殊方法的对象;
* `itertools.tee(it, n=2)`:产出一个由 n 个生成器组成的元组，每个生成器用于单独产出输入的可迭代对象中的元素;

备注：内置的 reversed 函数，是本节所述的函数中唯一一个不接受可迭代的对象，而只接受序列为参数的函数。这在情理之中，因为
reversed 函数从后向前产出元素，而只有序列的长度已知时才能工作。

In [41]:
list(itertools.groupby('LLLLAAGGG'))

[('L', <itertools._grouper at 0x20d6f7b5c18>),
 ('A', <itertools._grouper at 0x20d6f7b5eb8>),
 ('G', <itertools._grouper at 0x20d6f7b5128>)]

In [42]:
for char, group in itertools.groupby('LLLLAAAGG'): 
    print(char, '->', list(group))

L -> ['L', 'L', 'L', 'L']
A -> ['A', 'A', 'A']
G -> ['G', 'G']


In [43]:
animals = ['duck', 'eagle', 'rat', 'giraffe', 'bear','bat', 'dolphin', 'shark', 'lion']
animals.sort(key=len)
animals

['rat', 'bat', 'duck', 'bear', 'lion', 'eagle', 'shark', 'giraffe', 'dolphin']

In [44]:
for length, group in itertools.groupby(animals, len): 
    print(length, '->', list(group))

3 -> ['rat', 'bat']
4 -> ['duck', 'bear', 'lion']
5 -> ['eagle', 'shark']
7 -> ['giraffe', 'dolphin']


In [45]:
for length, group in itertools.groupby(reversed(animals), len):
    print(length, '->', list(group))

7 -> ['dolphin', 'giraffe']
5 -> ['shark', 'eagle']
4 -> ['lion', 'bear', 'duck']
3 -> ['bat', 'rat']


In [54]:
list(itertools.tee('abc'))

[<itertools._tee at 0x20d6ec344c8>, <itertools._tee at 0x20d6ec34908>]

In [66]:
g1,g2,g3=itertools.tee('abc',3)
list(zip(g1,g2,g3))

[('a', 'a', 'a'), ('b', 'b', 'b'), ('c', 'c', 'c')]

# with 语句和上下文管理器

```python
# create/aquire some resource
...
try:
    # do something with the resource
    ...
finally:
    # destroy/release the resource
    ...
```

处理文件，线程，数据库，网络编程等等资源的时候，我们经常需要使用上面这样的代码形式，以确保资源的正常使用和释放。

好在`Python` 提供了 `with` 语句帮我们自动进行这样的处理，例如之前在打开文件时我们使用： 

In [1]:
with open('my_file', 'w') as fp:
    # do stuff with fp
    data = fp.write("Hello world")

这等效于下面的代码，但是要更简便：

In [2]:
fp = open('my_file', 'w')
try:
    # do stuff with f
    data = fp.write("Hello world")
finally:
    fp.close()

## 上下文管理器

其基本用法如下：
```
with <expression>:
    <block>
```

`<expression>` 执行的结果应当返回一个实现了上下文管理器的对象，即实现这样两个方法，`__enter__` 和 `__exit__`：

In [3]:
print (fp.__enter__)
print (fp.__exit__)

<built-in method __enter__ of _io.TextIOWrapper object at 0x000000000567B558>
<built-in method __exit__ of _io.TextIOWrapper object at 0x000000000567B558>


`__enter__` 方法在 `<block>` 执行前执行，而 `__exit__` 在 `<block>` 执行结束后执行：

比如可以这样定义一个简单的上下文管理器：

In [4]:
class ContextManager(object):
    
    def __enter__(self):
        print ("Entering")
    
    def __exit__(self, exc_type, exc_value, traceback):
        print ("Exiting")

使用 `with` 语句执行：

In [5]:
with ContextManager():
    print ("  Inside the with statement")

Entering
  Inside the with statement
Exiting


即使 `<block>` 中执行的内容出错，`__exit__` 也会被执行：

In [6]:
with ContextManager():
    print (1/0)

Entering
Exiting


ZeroDivisionError: division by zero

## `__`enter`__` 的返回值

如果在 `__enter__` 方法下添加了返回值，那么我们可以使用 `as` 把这个返回值传给某个参数：

In [7]:
class ContextManager(object):
    
    def __enter__(self):
        print ("Entering")
        return "my value"
    
    def __exit__(self, exc_type, exc_value, traceback):
        print ("Exiting")

将 `__enter__` 返回的值传给 `value` 变量：

In [8]:
with ContextManager() as value:
    print (value)

Entering
my value
Exiting


一个通常的做法是将 `__enter__` 的返回值设为这个上下文管理器对象本身，文件对象就是这样做的：

In [9]:
fp = open('my_file', 'r')
print (fp.__enter__())
fp.close()

<_io.TextIOWrapper name='my_file' mode='r' encoding='cp936'>


In [10]:
import os
os.remove('my_file')

实现方法非常简单：

In [12]:
class ContextManager(object):
    
    def __enter__(self):
        print ("Entering")
        return self
    
    def __exit__(self, exc_type, exc_value, traceback):
        print ("Exiting")

In [13]:
with ContextManager() as value:
    print (value)

Entering
Exiting


## 错误处理

上下文管理器对象将错误处理交给 `__exit__` 进行，可以将错误类型，错误值和 `traceback` 等内容作为参数传递给 `__exit__` 函数：

In [14]:
class ContextManager(object):
    
    def __enter__(self):
        print ("Entering")
    
    def __exit__(self, exc_type, exc_value, traceback):
        print ("Exiting")
        if exc_type is not None:
            print ("  Exception:", exc_value)

如果没有错误，这些值都将是 `None`, 当有错误发生的时候：

In [15]:
with ContextManager():
    print( 1/0)

Entering
Exiting
  Exception: division by zero


ZeroDivisionError: division by zero

在这个例子中，我们只是简单的显示了错误的值，并没有对错误进行处理，所以错误被向上抛出了，如果不想让错误抛出，只需要将 `__exit__` 的返回值设为 `True`： 

In [16]:
class ContextManager(object):
    
    def __enter__(self):
        print ("Entering")
    
    def __exit__(self, exc_type, exc_value, traceback):
        print ("Exiting")
        if exc_type is not None:
            print ("  Exception:", exc_value)
            return True

In [17]:
with ContextManager():
    print( 1/0)

Entering
Exiting
  Exception: division by zero


在这种情况下，错误就不会被向上抛出。

# 装饰器及其使用

## 函数是一种对象

在 `Python` 中，函数是也是一种对象。

In [29]:
def foo(x):
    return x+1
    
print(type(foo))

<class 'function'>


In [30]:
# 查看函数拥有的方法：
dir(foo)

['__annotations__',
 '__call__',
 '__class__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__globals__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__kwdefaults__',
 '__le__',
 '__lt__',
 '__module__',
 '__name__',
 '__ne__',
 '__new__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

在这些方法中，`__call__` 是最重要的一种方法： 

In [31]:
foo.__call__(42)

43

相当于

In [32]:
foo(42)

43

因为函数是对象，所以函数可以作为参数传入另一个函数：

In [33]:
def  bar(f,x):
    x += 1
    return f(x)

In [34]:
bar(foo,1)

3

## 闭包与nonlocal

**闭包**指延伸了作用域的函数，其中包含在函数定义体里引用、但是不在定义体中定义的**非全局变量**。也就是说，它能访问定义体之外定义的非全局变量。

* 只有在涉及**嵌套函数**时才有闭包问题；
* 这里提到的非全局变量也就是指 嵌套作用域`enclosing scope`。


下面通过一个求平均值的例子来展示闭包。

In [29]:
# 计算移动平均值的高阶函数

def make_averager():
    series = []
    
    def averager(new_value):
        
        series.append(new_value)
        total = sum(series)
        return total/len(series)
    return averager

In [30]:
avg = make_averager()
avg(10)

10.0

In [31]:
avg(11)

10.5

在这个例子中，嵌套的部分就是闭包，见下。

```python
series = []
    
    def averager(new_value):
        
        series.append(new_value)
        total = sum(series)
        return total/len(series)
```

其中`series`就是我们说的自由变量/嵌套作用域/非全局变量，属于可以在嵌套函数体中引用但是定义在函数体外的变量。

In [32]:
## 审查make_averager
avg.__code__.co_varnames

('new_value', 'total')

In [34]:
avg.__code__.co_freevars

('series',)

In [35]:
avg.__closure__

(<cell at 0x000001CF8A375D38: list object at 0x000001CF8AADFD48>,)

In [36]:
avg.__closure__[0].cell_contents

[10, 11]

`series` 的绑定在返回的`avg.__closure__`属性中，里面的一个元素对应于`avg.__code__.co_freevars`一个名称。这些元素是`cell`对象，有个`cell_contents`属性，保存着真正的值，见上。

***
**nonloval**

上面求平均值的例子里，并不需要存储所有的历史值，只需要存储目前的总和以及元素个数。

In [37]:
# 计算移动平均值的高阶函数

def make_averager():
    count = 0
    total = 0
    
    def averager(new_value):
        count += 1
        total += new_value
        return total/count
    return averager

In [38]:
avg1 = make_averager()
avg1(10)

UnboundLocalError: local variable 'count' referenced before assignment

我们会发现，嵌套函数里找不到`count`变量的定义，这是因为`count`属于不可变类型，`count+=1`等价于`count=count+1`相当于在函数体内部对其赋值了，但是在函数体内并没有定义，所以报错。

为了解决这个问题，python3 引入了`nonlocal` 声明：

    把变量标记为自由变量，即使在函数中为变量赋予新值，也会变成自由变量。如果为`nonlocal`声明的变量赋予新值，闭包中保存的绑定也会更新。

In [39]:
# 计算移动平均值的高阶函数

def make_averager():
    count = 0
    total = 0
    
    def averager(new_value):
        nonlocal count,total
        
        count += 1
        total += new_value
        return total/count
    return averager

In [40]:
avg2 = make_averager()
avg2(10)

10.0

## 装饰器

装饰器是这样的一种函数，它接受一个函数作为输入，通常输出也是一个函数：

In [35]:
def dec(f):
    print ('I am decorating function', id(f))
    return f

将 `len` 函数作为参数传入这个装饰器：

In [36]:
declen = dec(foo)

I am decorating function 99603728


使用这个新生成的函数：

In [37]:
declen(10)

11

上面的例子中，我们仅仅返回了**函数的本身**，也可以利用这个函数**生成一个新的函数**，看一个新的例子：

In [14]:
def loud(f):
    def new_f(*arg1,**kw):
        print('calling with',arg1,kw)
        s = f(*arg1,**kw)
        print('return value is',s)
    return new_f

In [38]:
l = loud(foo)

In [41]:
l(12)

calling with (12,) {}
return value is 13


**用 @ 来使用装饰器**

```python
@dec
def foo(x):
    return x+1
```
`@`的含义是将`dec(foo)`这个复合函数简记为`foo`，但该函数的实际对应已经是`dec(f)`这个函数`return`的结果。

In [73]:
def foo(x):
    return x+1
    
def dec(f):
    print ('I am decorating function', id(f))
    def new_f1(x):
        print('it is a test')
        return f(x)
    return new_f1
    
foo = dec(foo)

I am decorating function 101874408


In [74]:
foo(4)

it is a test


5

可以修改为

In [75]:
@dec
def foo(x):
    return x+1

I am decorating function 99093224


In [76]:
foo(4)

it is a test


5

In [77]:
foo.__name__

'new_f1'

解释：

```python
@dec
def foo(x):
    return x+1
```

执行的过程等价于：

`dec(foo) = new_f1`

并且立即执行，出来了` print ('I am decorating function', id(f))`。

使用装饰器 `@dec`的意思是简记`foo=dec(foo)`

所以执行完上面的代码后，`foo`实际对应的函数为 `new_f1`，通过`foo.__name__`为`new_f1`也能看出来。

## 例子

### 例子1

定义两个装饰器，一个将原来的函数值加一，另一个乘二：

In [82]:
def plus_one(f):
    def f1(x):
        return f(x) + 1
    return f1

def times_two(f):
    def f2(x):
        return f(x) * 2
    return f2

In [83]:
@plus_one
@times_two
def goo(x):
    return int(x)

In [84]:
goo('2')

5

记$f_1 = f+ 1;f_2=f*2$，以上执行过程对应的数学表达为：

$$plus\_one(time\_two(goo)) \\
= plus\_one(f_2) \\
= f_1(f_2) \\
= f_1(f*2) \\
= f*2+1$$

本质上是一个复合函数。

### 自动添加函数

In [3]:
promos =  []

def promotion(promo_func):
    promos.append(promo_func)
    return promo_func

@promotion
def add(x,y):
    return x+y

@promotion
def diff(x,y):
    return x-y

@promotion
def pro(x,y):
    return x*y

In [4]:
promos

[<function __main__.add(x, y)>,
 <function __main__.diff(x, y)>,
 <function __main__.pro(x, y)>]

装饰器作为方案的优点：

1. 被添加的函数不需要起特殊的名称来专门定义函数来搜寻符合的函数；
2. `@promotion`突出了被装饰的函数的作用，还便于临时禁用某个函数，只需要把装饰器注释掉；
3. 函数可以在其他模块中定义，系统中任何地方都可以，只要使用了`@promotion`即可。

### 注册一个函数

来看这样的一个例子，定义一个类：

In [125]:
class Registry(object):
    def __init__(self):
        self._data = {}
    def register(self, f, name=None):
        if name == None:
            name = f.__name__
        self._data[name] = f

`register` 方法接受一个函数，将这个函数名作为属性注册到对象中。

产生该类的一个对象：

In [121]:
registry = Registry()

使用该对象的 `register` 方法作为修饰符：

In [122]:
@registry.register
def greeting():
    print ("hello world")

这样这个函数就被注册到 `registry` 这个对象中去了：

In [123]:
registry._data

{'greeting': <function __main__.greeting>}

## 标准库中的装饰器

### functools.wraps

In [126]:
def logging_call(f):
    def wrapper(*a, **kw):
        print ('calling {}'.format(f.__name__))
        return f(*a, **kw)
    return wrapper

@logging_call
def square(x):
    '''
    square function.
    '''
    return x ** 2

print (square.__doc__, square.__name__)

None wrapper


我们使用修饰符之后，`square` 的 `metadata` 完全丢失了，返回的函数名与函数的 `docstring` 都不对。

一个解决的方法是从 `functools` 模块导入 `wraps` 修饰符来修饰我们的修饰符：

In [127]:
import functools

def logging_call(f):
    @functools.wraps(f)
    def wrapper(*a, **kw):
        print ('calling {}'.format(f.__name__))
        return f(*a, **kw)
    return wrapper

@logging_call
def square(x):
    '''
    square function.
    '''
    return x ** 2

print (square.__doc__, square.__name__)


    square function.
     square


### functools.lru_cache

`functools.lru_cache`是非常实用的装饰器，实现了备忘功能，是一项优化技术，把耗时的函数结果保存起来，避免传入相同参数时重复计算。LRU是`Least Recently Used`的缩写，表明缓存不会无限制增长，一段时间不用的缓存条目会被扔掉。

```python 
functools.lru_cache(maxsize=128,typed=False):
```
1. `maxsize`参数存储多少个调用的结果，缓存满了之后，旧结果会被扔掉，腾出空间，为了得到最佳性能，应该设为2的幂；

2. `type`参数如果设定为True,把不同参数类型得到的结果分开保存。

3. `lru_cache`使用字典存储结果，而且键根据调用时传入的定位参数和关键字参数创建，所以被`lru_cache`装饰的函数，它的所有参数都必须是**可散列的**。

生成第n个斐波那契数这种慢速递归函数适合用`lru_cache`，如下示例：

In [11]:
# 计时器
import time
import functools

def clock(func):
    @functools.wraps(func)
    def clocked(*args,**kwargs):
        t0=time.time()
        result=func(*args,**kwargs)
        elapsed = time.time()-t0
        name=func.__name__
        arg_lst = []
        if args:
            arg_lst.append(', '.join(repr(arg) for arg in args))
        
        if kwargs:
            pairs = ['%s=%r'%(k,w) for k,w in sorted(kwargs.items())]
            arg_lst.append(', '.join(pairs))
        
        arg_str = ', '.join(arg_lst)
        print('[%0.10fs] %s(%s) -> %r'%(elapsed,name,arg_str,result))
        return result
    return clocked    

生成第n个斐波那契数，递归方式非常耗时

In [12]:
@clock
def fibonacci(n):
    if n<2:
        return n
    else:
        return fibonacci(n-2) + fibonacci(n-1)

In [13]:
print(fibonacci(6))

[0.0000000000s] fibonacci(0) -> 0
[0.0000000000s] fibonacci(1) -> 1
[0.0000000000s] fibonacci(2) -> 1
[0.0000000000s] fibonacci(1) -> 1
[0.0000000000s] fibonacci(0) -> 0
[0.0000000000s] fibonacci(1) -> 1
[0.0000000000s] fibonacci(2) -> 1
[0.0000000000s] fibonacci(3) -> 2
[0.0000000000s] fibonacci(4) -> 3
[0.0000000000s] fibonacci(1) -> 1
[0.0000000000s] fibonacci(0) -> 0
[0.0000000000s] fibonacci(1) -> 1
[0.0000000000s] fibonacci(2) -> 1
[0.0000000000s] fibonacci(3) -> 2
[0.0000000000s] fibonacci(0) -> 0
[0.0000000000s] fibonacci(1) -> 1
[0.0000000000s] fibonacci(2) -> 1
[0.0000000000s] fibonacci(1) -> 1
[0.0000000000s] fibonacci(0) -> 0
[0.0000000000s] fibonacci(1) -> 1
[0.0010004044s] fibonacci(2) -> 1
[0.0010004044s] fibonacci(3) -> 2
[0.0010004044s] fibonacci(4) -> 3
[0.0010004044s] fibonacci(5) -> 5
[0.0010004044s] fibonacci(6) -> 8
8


浪费时间的地方在于：`fibonacci(1)`被调用了8次，...但是，如果增加两行代码，使用`lru_cache`性能会显著改善：

In [14]:
@functools.lru_cache()
@clock
def fibonacci(n):
    if n<2:
        return n
    else:
        return fibonacci(n-2) + fibonacci(n-1)

In [15]:
print(fibonacci(6))

[0.0000000000s] fibonacci(0) -> 0
[0.0000000000s] fibonacci(1) -> 1
[0.0000000000s] fibonacci(2) -> 1
[0.0000000000s] fibonacci(3) -> 2
[0.0000000000s] fibonacci(4) -> 3
[0.0000000000s] fibonacci(5) -> 5
[0.0000000000s] fibonacci(6) -> 8
8


这里叠放了装饰器：`lru_cache()`应用到`@clock`返回的函数上。

### functools.singledispatch

In [16]:
# 我们有下面这样一个生成HTML的函数

import html

def htmlize(obj):
    content = html.escape(repr(obj))
    return '<pre>{}</pre>'.format(content)

这个函数适用于任何数据类型，但是现在我们想做一个扩展，让它用特别的方式显示某些类型：

1. `str`: 把内部的换行符替换为`<br>\n`;不使用`<pre>`而是`<p>`;

2. `int`:以十进制和十六进制显示数字；

3. `list`:输出一个html列表，根据各个元素的类型进行格式化

如果要实现以上三种变形，一种常见的做法是把`htmlize`函数变成一个分派函数，使用一串`if/elif/elif`，调用专门的函数，如`htmlize_str、htmlize_int`等。

这样实现的缺点是不便于用户扩展，显得笨拙；时间一长，分派函数会变得很大，而且它与各个专门函数之间的耦合性也很紧密。

`python3.4`新增的`functools.singledispatch`装饰器可以把整个方案拆分成多个模块，甚至可以为你无法修改的类提供专门的函数。使用`@singledispatch`装饰的普通函数会变成泛函数：根据第一个参数的类型，以不同方式执行相同操作的一组函数。

使用`singledispatch`创建一个自定义的`htmlize.register`装饰器，把多个函数绑在一起组成一个泛函数：

In [27]:
from functools import singledispatch
from collections import abc
import numbers
import html

@singledispatch
def htmlize(obj):
    content = html.escape(repr(obj))
    return '<pre>{}</pre>'.format(content)

@htmlize.register(str)
def _(text):
    content = html.escape(text).replace('\n','<br>\n')
    return '<p>{}</p>'.format(content)

@htmlize.register(numbers.Integral)
def _(n):
    return '<pre>{0} (0x{0:x})</pre>'.format(n)


@htmlize.register(tuple)
@htmlize.register(abc.MutableSequence)
def _(seq):
    inner = '</li>\n<li>'.join(htmlize(item) for item in seq)
    return '<ur>\n<li>' + inner + '</li>\n</ur>'

In [28]:
htmlize('it is a \n test')

'<p>it is a <br>\n test</p>'

In [29]:
htmlize(10)

'<pre>10 (0xa)</pre>'

In [30]:
htmlize([10,'it is a \n test'])

'<ur>\n<li><pre>10 (0xa)</pre></li>\n<li><p>it is a <br>\n test</p></li>\n</ur>'

只要可能，注册的专门函数应该处理抽象基类(如`abc.MutableSequence`以及`numbers.Integral`),不要处理具体实现(如`list`以及`int`)，这样代码支持的兼容类型更广泛。

`@singledispatch`的优点是支持模块化扩展，各个模块可以为它想要支持的各个类型注册一个专门函数。

### classmethod 修饰符

在 `Python` 标准库中，有很多自带的修饰符，例如 `classmethod` 将一个对象方法转换了类方法： 

In [105]:
class Foo(object):
    @classmethod
    def bar(ss,x,y):
        print( 'the input is {} and {}'.format(x,y))
        
    def __init__(self):
        pass

类方法可以通过 `类名.方法` 来调用：

In [106]:
Foo.bar(12,13)

the input is 12 and 13


将函数方法中的`ss`代替为`self`

### Numpy 的 @vectorize 修饰符

`numpy` 的 `vectorize` 函数将一个函数转换为 `ufunc`，事实上它也是一个修饰符。

ufunc函数: ufunc是universal function的缩写，它是一种能对数组的每个元素进行操作的函数。NumPy内置的许多ufunc函数都是在C语言级别实现的，因此它们的计算速度非常快。

In [107]:
from numpy import vectorize, arange

@vectorize
def f(x):
    if x <= 0:
        return x
    else:
        return 0

f(arange(-10.0,10.0))

array([-10.,  -9.,  -8.,  -7.,  -6.,  -5.,  -4.,  -3.,  -2.,  -1.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.])

等价于

In [112]:
def f(x):
    if x <= 0:
        return x
    else:
        return 0
v1 = vectorize(f)

## 参数化装饰器

如何让装饰器接受除了被装饰的函数以外的其他参数呢？答案是，装饰器工厂函数：
    
        把参数传给它，返回一个装饰器，然后把它应用到要装饰的函数上。

### 一个参数化的装饰器

In [5]:
registry = []

def regsiter(func):
    print('running register(%s)'% func)
    registry.append(func)
    return func

@regsiter
def f1():
    print('running f1()')


running register(<function f1 at 0x00000000061E56A8>)


为了便于启用或者禁用register执行的函数注册功能，可以想办法为它提供一个可选的active参数，设为false时，不注册被装饰的函数。

从概念上看，这个新的register函数不是装饰器，而是装饰器工厂函数。调用它会返回真正的装饰器，这才是应用到目标函数上的装饰器。

In [9]:
registry=set()

def register(active=True):
    def decorate(func):
        print('running register(active=%s)->decorate(%s)'%(active,func))
        if active:
            registry.add(func)
        else:
            registry.discard(func)
            
        return func
    return decorate

@register(active=False)
def f1():
    print('running f1()')

@register()
def f2():
    print('running f2()')
    
def f3():
    print('running f3()')

running register(active=False)->decorate(<function f1 at 0x00000000065CB268>)
running register(active=True)->decorate(<function f2 at 0x00000000065CB510>)


In [10]:
registry

{<function __main__.f2>}

注意：

1. decorate才是真正的装饰器；
2. regsiter是一个装饰器工厂函数，返回decorate;
3. 即使不传入参数，register也必须作为函数调用(`@register()`),即要返回真正的装饰器decorate;

如果不使用`@`,就要像常规函数那样使用register，如果想把f3添加到registry中，句法如下：

In [11]:
register()(f3)

running register(active=True)->decorate(<function f3 at 0x00000000065CB2F0>)


<function __main__.f3>

In [12]:
registry

{<function __main__.f2>, <function __main__.f3>}

### 参数化clock装饰器

再次讨论clock装饰器，为它添加一个功能：让用户传入一个格式字符串，控制装饰器函数的输出。

In [13]:
# 计时器
import time
import functools

def clock(func):
    def clocked(*args):
        t0=time.time()
        result=func(*args)
        elapsed = time.perf_counter()-t0
        name=func.__name__
        arg_str=', '.join(repr(arg) for arg in args)
        print('[%0.8fs] %s(%s) -> %r'%(elapsed,name,arg_str,result))
        return result
    return clocked    

我们对上面这个装饰器进行改造，通过装饰器工厂来实现想要添加的功能

In [19]:
default_fmt = '[{elapsed:0.8f}s]{name}{args} -> {result}'

def clock(fmt=default_fmt):
    def decorate(func):
        def clocked(*_args):
            t0=time.time()
            _result=func(*_args)
            elapsed = time.perf_counter()-t0
            name=func.__name__
            args=', '.join(repr(arg) for arg in _args)
            result = repr(_result)
            print(fmt.format(**locals()))
            return _result
        return clocked
    return decorate

In [20]:
@clock()
def snooze(seconds):
    time.sleep(seconds)

for i in range(3):
    snooze(.123)

[-1583162030.91540956s]snooze0.123 -> None
[-1583162030.91539884s]snooze0.123 -> None
[-1583162030.91541076s]snooze0.123 -> None


In [21]:
@clock('{name}: {elapsed}s')
def snooze(seconds):
    time.sleep(seconds)

for i in range(3):
    snooze(.123)

snooze: -1583162030.918761s
snooze: -1583162030.9187465s
snooze: -1583162030.918679s


In [22]:
@clock('{name}(args) dt={elapsed:0.3f}s')
def snooze(seconds):
    time.sleep(seconds)

for i in range(3):
    snooze(.123)

snooze(args) dt=-1583162030.922s
snooze(args) dt=-1583162030.922s
snooze(args) dt=-1583162030.921s


以上示例就是我们通过装饰器工厂实现了用户可以传入不同的打印格式参数的功能

注：装饰器最好通过实现 `__call__`方法的类来实现，不应该像上面介绍的通过函数实现，此处使用函数只是为了便于理解。